![](https://verstaresearch.com/wp-content/uploads/2014/08/VB2014-08-27-01-900x450.jpg)

# 1. Intro
**Data cleaning** is such an integral part of data analysis.Unlike here in Kaggle,almost all data you see in the real world would be dirty and messy. Some even say data cleaning would take 80% of data analysis time.
   
### What we will do in this notebook
- Data cleaning
- Basic EDA
- Visualization

### This notebook is stil under construction.
### Feel free to <font color=deepskyblue> FORK  </font> this notebook, Please  <font color=deepskyblue> UPVOTE !! </font> if it's helpful to you  <font color=deepskyblue> : ) </font>


>    By the way,english is not my native language, so dont freak out if you run into any language error. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 2. Know your data
The second step or I would say the very fisrt step of any given data analysis project would be getting to know your data especially when you are dealing a messy one.

Take a look at our data as below, the data is quite obscure,it's hard to understand for a human,not mentioned for a computer.in this kind of situation,you have ways to get acquaintance with your data as follows:

1. Go to the data source page [WHO OBESITY DATA](https://apps.who.int/gho/data/node.main.A900A?lang=en) 
2. Read the description in this kaggle data page [Obesity among adults by country, 1975-2016](https://www.kaggle.com/amanarora/notebook)
3. If solution 1 or 2 doesnnot work or hard to do, you can always go to ask data curator directly.

In [ ]:
data=pd.read_csv('/kaggle/input/obesity-among-adults-by-country-19752016/data.csv')
data.head()

# 3.Why and how data is messy?
In most cases, data is collected by human or machines,a tiny glich would cause a long strip of bad data.if the data is collected by human, then it is a big chance that it would be messy. data can be dirty in many different ways,but mostly fall into those categories :

1. **Missing data**<br>like NAN
2. **Validity of data**<br>like 2016.1 / 2016.2 in the column
3. **Outliers**<br>like if a BMI entry is greater than 100
4. **Consistency of data**<br>the unit of every entry is not the same
5. **Correctness of data**<br>
    we are not gonna go through this ,but it is an important part of doing analysis in bussiness world,
    basically you need external data source or database to cross check the data in your hand because as we always say:
>  You dont know what you dont know 

6. Data is in **wide form** not in long form <br>
    we are gonna go deeper about this one.

Are you ready, it's time to get our hands dirty!

# 4.1 long form VS wide form
   The original data we have here is in wide form which means the form is very wide literally.
   
   After read the description in WHO webpage we know that the year value 2016 or 2017 are actually values but were put in column which made the form wide.
   
   the .1 .2 in year number stand for gender, we gonna fix that later. 
   
   Wide data is not easy to analyze or stored effectively in computer, so we want to change it as soon as we can.
   go to read this [tidy-data](https://vita.had.co.nz/papers/tidy-data.pdf) if you want to know more.

In [ ]:
df=data.copy()
df.rename(columns={'Unnamed: 0':'country'},inplace=True)
df=df.melt(id_vars=['country'],var_name='year')

In [ ]:
df

# 4.2 correct the format

1. we will dorp the first 3 row since its actually headers in the original forms.
2. correct year value
3. correct the gender value

In [ ]:
df=df.drop([0,1,2]).reset_index()
df[['year','gender']]=df['year'].str.split('.',expand=True)
df=df.drop(columns=['index'])
df

In [ ]:
df['gender']=df['gender'].map({None:'both sex','1':'male','2':'female'})
df

4. correct the BMI value columns

   from the webpage of WHO we can know that the values in [] are actually estimation intervel，so we need to seperate them into 3 columns
   
   you can use **str.matach()** or **str.findall()** with **regular expression** to extract float number in this field,but I am gonna use **str.split()**

In [ ]:
df.rename(columns={'value':'BMI'},inplace=True)
df1=df.copy()
df['BMI']=df1['BMI'].str.split('[',expand=True)[0]
df['BMI_lower_esti']=df1['BMI'].str.split('[',expand=True)[1].str.split('-',expand=True)[0]
df['BMI_upper_esti']=df1['BMI'].str.split('[',expand=True)[1].str.split('-',expand=True)[1].str.split(']',expand=True)[0]
df=df[['country','year','gender','BMI','BMI_upper_esti','BMI_lower_esti']] #reset the column sequence
df

# 4.3 check validity of all columns or fields

We now have a pretty clean data compared to the one we just got.
but our job is still not done yet.
we need to go through every columns or fields to make sure the data is reletively correct.

### 4.3.1 Country columns

**What we know：**

1. There is a country named **country** which need to be fixed 
2. There are **Nones** in country column which need to be fixed 
3. We have 

**What we do：**

We gonna drop those entries.

In [ ]:
#locate Na
df.isna().sum()

In [ ]:
#delet country == 'Country' and country == None entries

df1=df.dropna(subset=['country'])
df1=df1.drop(df[df.country=='Country'].index)

### 4.3.2 BMI \ BMI_upper_esti and BMI_lower_esti columns

**What we know：**

1. 4 contries have no BMI data which are Monaca,Sudan,South Sudan and San Marino,hence they dont have estimations.
2. We have 191 countries that do have BMI data and each of them has 126 entries. 
3. The descriptive statistics of BMI data seems OK, no outliers.

**What we do：**

1. We gonna create a new dataframe without those 4 countries to analyze.
2. We gonna change the data type of BMI and estimations to float. 

In [ ]:
print(df1.isna().sum())
print('\n')
print(df1[df1.BMI_upper_esti.isna()|df1.BMI_lower_esti.isna()].BMI.value_counts())
print('\n')
print(df1[df1.BMI=='No data'].country.value_counts())
con=df1[df1.BMI=='No data'].country.value_counts().index

from above we can find that thare are 4 countries have no data, so exclude them, we have df2 to investigare 

In [ ]:
df2=df1[~df1.country.isin(con)]
# df2 dataframe is the one excluded no data countries

In [ ]:
print(df2.country.value_counts().index)
val=df2.country.value_counts().values
np.unique(val)

In [ ]:
df2[['BMI','BMI_upper_esti','BMI_lower_esti']]=df2.loc[:,['BMI','BMI_upper_esti','BMI_lower_esti']].astype('float')
#change datatype to float
df2.loc[:,['BMI','BMI_upper_esti','BMI_lower_esti']].describe()

### 4.3.3 Year and gender columns

**What we know：**

1. We have year across from 1975 to 2016

In [ ]:
df2.year.value_counts().sort_index()
# year column seems ok

In [ ]:
df2.gender.value_counts()
#gender columns seems ok

### Clean process finished

We now have a pretty clean data, it's time to do some EDA and visualization

<img src='https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/120/apple/237/clapping-hands-sign_1f44f.png' align='left'/>

# 5 visualization and EDA

Before you doing any EDA, come up with some questions first. Question orientated is always a good way to explore a set of data, you could easily fall into rabbit holes you enconter along the process otherwise.You dont have to write those question down, I'm just gonna do it.

By the way, EDA means **Exploratory Data Analysis**

**What question we could possibly answer through this data?**

1. which countries have the highest/lowest BMI ?
2. How do those country change over the past few years ?
3. How does the BMI change globally ?
4. If there is any diffrence in obesity between male and female ?

In [ ]:
df2.head()

In [ ]:
df_mean_BMI=df2.groupby(by=['year','gender']).BMI.mean().reset_index().set_index('year')
df_median_BMI=df2.groupby(by=['year','gender']).BMI.median().reset_index().set_index('year')

## Mean/Median BMI Changes over the years 

**What we can know from the graph:**

1. The mean/median BMI value across the globe is steadily increase over the last 41 years for both male and female

In [ ]:
fig = plt.figure(figsize=(16,10))
ax = fig.add_subplot(211)
g=sns.lineplot(data=df_mean_BMI,x=df_mean_BMI.index,y='BMI',hue='gender',ax=ax)
ax.tick_params(axis='x', rotation=45)
plt.title('BMI Changes over the years',size=20)
ax1 = fig.add_subplot(212)
g1=sns.lineplot(data=df_median_BMI,x=df_median_BMI.index,y='BMI',hue='gender',ax=ax1)
ax1.tick_params(axis='x', rotation=45)
ax.set_ylabel('BMI-mean',size=20)
ax1.set_ylabel('BMI-median',size=20)
plt.show()

## BMI distribution in 1975 1995 and 2016

**What we can know from the graph:**

1. In year 1975,the distribution is unimodal, left-skrewed clearly.
2. In year 1995 and 2016,the center of distribution is moving towards right over time and become multimodal(two peaks on the curve)

In [ ]:
fig = plt.figure(figsize=(12,6))
sns.distplot(df2[df2.year=='1975'].BMI,hist=False,label='1975')
sns.distplot(df2[df2.year=='1995'].BMI,hist=False,label='1995')  # 1995 is roughly the middle point between 1975 and 2016
sns.distplot(df2[df2.year=='2016'].BMI,hist=False,label='2016')
plt.title('BMI Distribution Across the Globe',size=20)
plt.show()

## How top 10 BMI countries changes

They didn't change much

In [ ]:
df3=df2[df2.gender=='both sex'].sort_values('BMI',ascending=False).groupby('year').head(10).sort_values('year')
df3 #Top contries from 1975 to 2016

In [ ]:
years=df3.year.unique().astype('int')
year=[x for x in years if x%2==0 ] #sampling years using systematic sampling
df4=df3[df3.year.astype('int').isin(year)].sort_values(by=['year','BMI'])
df4.country=df4.country.str.replace('Federated States of','FSo')

In [ ]:
grid = sns.FacetGrid(df4, col="year",col_wrap=4,sharey=False)
grid.map(plt.barh,"country", "BMI")
plt.suptitle('Top 10 countries over 41 years',x=0.5,y=1.02,size=25)
plt.show()

## Diffrence between male and famale

In [ ]:
data_temp=df2[(df2.gender!='both sex')&df2.year.isin(['1975','1985','1995','2005','2015'])]
plt.figure(figsize=(10,8))
sns.violinplot(data=data_temp,x='year',y='BMI',hue='gender',palette='Accent',split=True)
plt.title('Obesity Distribution between male and female',size=20)
plt.legend(loc='upper left')
plt.show()

 I'm still trying to find better ways to explore and visualize the difference between male and female across countries in long term.
if you have better ideas, let me know in the comment,plz.

## A basic Racing Charts

In [ ]:
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
df3.year=df3.year.astype('int')

In [ ]:
def draw_barchart(year):
    dff = df3[df3['year'].eq(year)].sort_values(by='BMI', ascending=True).tail(10)
    ax.clear()
    ax.barh(dff['country'], dff['BMI'],color='lightblue')
    dx = dff['BMI'].max() / 200
    for i, (value, name) in enumerate(zip(dff['BMI'], dff['country'])):
        ax.text(value-dx, i,     name,           size=14, weight=600, ha='right', va='center')
        #ax.text(value-dx, i-.25, group_lk[name], size=10, color='#444444', ha='right', va='baseline')
        ax.text(value+dx, i,     f'{value:,.0f}',  size=14, ha='left',  va='center')
    # ... polished styles
    ax.text(1, 0.4, year, transform=ax.transAxes, color='#777777', size=46, ha='right', weight=800)
    ax.text(0, 1.06, 'Obesity %', transform=ax.transAxes, size=12, color='#777777')
    ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
    ax.xaxis.set_ticks_position('top')
    ax.tick_params(axis='x', colors='#777777', labelsize=12)
    ax.set_yticks([])
    ax.margins(0, 0.01)
    ax.grid(which='major', axis='x', linestyle='-')
    ax.set_axisbelow(True)
    ax.text(0, 1.12, 'The most Obesity% countries in the world from 1975 to 2016',
            transform=ax.transAxes, size=24, weight=600, ha='left')
    #ax.text(1, 0, 'by @pratapvardhan; credit @jburnmurdoch', transform=ax.transAxes, ha='right',color='#777777', bbox=dict(facecolor='white', alpha=0.8, edgecolor='white'))

In [ ]:
%%capture
fig, ax = plt.subplots(figsize=(18, 10))
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1975, 2016))

In [ ]:
HTML(animator.to_jshtml())
# or use animator.to_html5_video() or animator.save()

# To be continued...

This notebook is still under construction, so if you want to know anything about data cleaning or EDA process,just leave your commet below,and upvote it if you think it's helpful.